<a href="https://colab.research.google.com/github/mishra0488/TestAssignment/blob/master/MyFirstGoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This program uses artificial recurrent neural network
# to predict closing stock price of a corporation

In [1]:
# Import libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# Get stock code

In [17]:
df = web.DataReader('AAPL', data_source='yahoo', start='01-01-2012', end='20-03-2021')
# Show data
df.shape # get number of row col

(2318, 6)

In [ ]:
# visualize the closing price history
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.show()

In [18]:
# create a new dataframe with only the close column
data = df.filter(['Close'])
# convert the dataframe to a numpy error
dataset = data.values
# Get the number of rows to train the LSTM maodel
training_data_len = math.ceil(len(dataset)*.8)  # ceil to round up and .8 for 80% training set
training_data_len

1855

In [ ]:
# scale the data
scaler = MinMaxScaler(feature_range=(0, 1)) # Scale the data between 0 and 1
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [13]:
# Create the training dataset
# Create the scaled training data set
train_data = scaled_data[0:training_data_len, :]
# split the data into x_train and Y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
  x_train.append(train_data[i-60:i, 0])
  y_train.append(train_data[i, 0])
  if i<=60:
    print(x_train)
    print(y_train)
    print()

[array([0.00572147, 0.00633231, 0.00760099, 0.00880885, 0.00862367,
       0.00904103, 0.00885031, 0.00852969, 0.00809297, 0.00944457,
       0.0106635 , 0.01028759, 0.00822841, 0.01019361, 0.00825882,
       0.01551431, 0.01495322, 0.01568568, 0.01726944, 0.01822855,
       0.01814839, 0.01785265, 0.01911303, 0.02029878, 0.02164209,
       0.02381183, 0.02836966, 0.02843875, 0.03097611, 0.0328722 ,
       0.02961346, 0.03086831, 0.03084343, 0.034362  , 0.03386171,
       0.03478765, 0.03645157, 0.03737752, 0.04004478, 0.04198786,
       0.04254895, 0.0427452 , 0.03942288, 0.03862131, 0.03874017,
       0.04186349, 0.04274244, 0.04463025, 0.04908028, 0.05501734,
       0.05390621, 0.05390897, 0.05820147, 0.05954478, 0.05858842,
       0.05771501, 0.05680565, 0.05982669, 0.0618997 , 0.06276759])]
[0.06062272301844436]



In [14]:
# convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [15]:
# Reshape the data (currently the data is in 2d shape so this is needed to convert the data into 3d(expected for neural network))
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(1795, 60, 1)

In [19]:
# build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [20]:
# compile the modedl
model.compile(optimizer='adam', loss='mean_squared_error')

In [21]:
# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

1795/1795 [==============================] - 38s 20ms/step - loss: 6.2609e-04


In [22]:
# Create the testing data set
test_data = scaled_data[training_data_len - 60: , :]
# create the data set x_test, y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i,0])

In [25]:
# convert the data to a numpy array
x_test = np.array(x_test)